# Wall Street Ortho - Bronze Layer Data Ingestion

This notebook ingests raw Wall Street Orthodontics data into the bronze layer of our data warehouse.

## Architecture Context
- **Bronze Layer**: Raw data with minimal transformation
- **Target Schema**: `bronze_ops` for operational data
- **Source**: CSV files from `/data/wall_street_ortho/`

## Data Files
1. `aging_raw_021026.csv` - Patient aging/billing data
2. `appoints_raw_021026.csv` - Appointment data  
3. `patients_raw_021026.csv` - Patient demographics and status
4. `production_raw_021026.csv` - Financial production/ledger data
5. `referrals_raw_021026.csv` - Referral source data
6. `treatment_raw_021026.csv` - Treatment history data

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path

# Add utils to path
sys.path.append('../utils')
from connect_db import get_engine

# Set paths
data_dir = Path('../data/wall_street_ortho')
print(f"Data directory: {data_dir.absolute()}")
print(f"Files available: {list(data_dir.glob('*.csv'))}")

Data directory: /Users/andylee/Desktop/projects/scrub_network/supabase_setup/src/../data/wall_street_ortho
Files available: [PosixPath('../data/wall_street_ortho/referrals_raw_021026.csv'), PosixPath('../data/wall_street_ortho/treatment_raw_021026.csv'), PosixPath('../data/wall_street_ortho/aging_raw_021026.csv'), PosixPath('../data/wall_street_ortho/production_raw_021026.csv'), PosixPath('../data/wall_street_ortho/patients_raw_021026.csv'), PosixPath('../data/wall_street_ortho/appoints_raw_021026.csv')]


In [2]:
# Examine CSV headers to understand the raw column structure
def examine_csv_headers(file_path):
    """Read first few rows and examine headers"""
    df = pd.read_csv(file_path, nrows=3)
    print(f"\n{'='*60}")
    print(f"FILE: {file_path.name}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape}")
    print(f"Columns ({len(df.columns)}):")
    for i, col in enumerate(df.columns):
        print(f"  {i:2d}: {repr(col)}")
    print(f"\nFirst row sample:")
    for col in df.columns[:5]:  # Show first 5 columns only
        print(f"  {col}: {repr(df[col].iloc[0])}")
    return df.columns.tolist()

# Examine all CSV files
csv_files = sorted(data_dir.glob('*.csv'))
all_headers = {}

for csv_file in csv_files:
    if csv_file.name != '.DS_Store':
        all_headers[csv_file.name] = examine_csv_headers(csv_file)


FILE: aging_raw_021026.csv
Shape: (3, 13)
Columns (13):
   0: "Patient's ID (Guid)"
   1: 'All Accounts - 0-30 Days'
   2: 'All Accounts - 31-60 Days'
   3: 'All Accounts - 61-90 Days'
   4: 'All Accounts - Over 90 Days'
   5: 'All Accounts - Account Balance'
   6: 'All Accounts - Credits'
   7: 'All Accounts - Insurance Company Name'
   8: 'All Accounts - Past Due'
   9: 'All Accounts - Total Due'
  10: 'All Accounts - Unbilled'
  11: 'All Accounts - Patient Insurance ID (GUID)'
  12: "Patient's Name (First Last)"

First row sample:
  Patient's ID (Guid): '{EE314011-2837-4466-BDA0-E50717731E6D}'
  All Accounts - 0-30 Days: np.int64(462)
  All Accounts - 31-60 Days: np.int64(458)
  All Accounts - 61-90 Days: np.int64(458)
  All Accounts - Over 90 Days: np.int64(2290)

FILE: appoints_raw_021026.csv
Shape: (3, 42)
Columns (42):
   0: 'Unnamed: 0'
   1: 'Unnamed: 1'
   2: 'Unnamed: 2'
   3: 'Unnamed: 3'
   4: 'Unnamed: 4'
   5: 'Confirmed'
   6: 'Message'
   7: 'Minutes'
   8: 'Start tim

In [3]:
# File configuration - which header row to use for each file
file_config = {
    'aging_raw_021026.csv': 0,          # Standard header
    'appointments_raw_021026.csv': 1,    # Skip first row
    'patients_raw_021026.csv': 0,        # Standard header  
    'production_raw_021026.csv': 1,      # Skip first row
    'referrals_raw_021026.csv': 0,       # Standard header
    'treatment_raw_021026.csv': 0        # Standard header
}

def examine_csv_with_correct_headers(file_path):
    """Read CSV with the correct header row"""
    filename = file_path.name
    header_row = file_config.get(filename, 0)
    
    print(f"\n{'='*60}")
    print(f"FILE: {filename} (using header row {header_row})")
    print(f"{'='*60}")
    
    try:
        df = pd.read_csv(file_path, header=header_row, nrows=3)
        print(f"Shape: {df.shape}")
        print(f"Columns ({len(df.columns)}):")
        for i, col in enumerate(df.columns):
            print(f"  {i:2d}: {repr(col)}")
        print(f"\nFirst row sample:")
        for col in df.columns[:5]:  # Show first 5 columns only
            if not df.empty:
                print(f"  {col}: {repr(df[col].iloc[0])}")
        return df.columns.tolist()
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return []

# Examine all CSV files with correct headers
csv_files = sorted(data_dir.glob('*.csv'))
corrected_headers = {}

for csv_file in csv_files:
    if csv_file.name != '.DS_Store' and csv_file.name in file_config:
        corrected_headers[csv_file.name] = examine_csv_with_correct_headers(csv_file)


FILE: aging_raw_021026.csv (using header row 0)
Shape: (3, 13)
Columns (13):
   0: "Patient's ID (Guid)"
   1: 'All Accounts - 0-30 Days'
   2: 'All Accounts - 31-60 Days'
   3: 'All Accounts - 61-90 Days'
   4: 'All Accounts - Over 90 Days'
   5: 'All Accounts - Account Balance'
   6: 'All Accounts - Credits'
   7: 'All Accounts - Insurance Company Name'
   8: 'All Accounts - Past Due'
   9: 'All Accounts - Total Due'
  10: 'All Accounts - Unbilled'
  11: 'All Accounts - Patient Insurance ID (GUID)'
  12: "Patient's Name (First Last)"

First row sample:
  Patient's ID (Guid): '{EE314011-2837-4466-BDA0-E50717731E6D}'
  All Accounts - 0-30 Days: np.int64(462)
  All Accounts - 31-60 Days: np.int64(458)
  All Accounts - 61-90 Days: np.int64(458)
  All Accounts - Over 90 Days: np.int64(2290)

FILE: appointments_raw_021026.csv (using header row 1)
Shape: (3, 42)
Columns (42):
   0: "Patient's ID"
   1: "Patient's ID (Guid)"
   2: 'Appointment Created By'
   3: 'Appointment Created Date'
  

In [4]:
# Column mappings from raw CSV headers to bronze table columns
# This maps the CSV column names to our standardized bronze layer column names

def get_column_mappings():
    """Define column mappings for each file"""
    
    # aging_raw_021026.csv → bronze_fin.aging_accounts_raw_wso
    aging_mapping = {
        "Patient's ID (Guid)": "patient_id_guid",
        "All Accounts - 0-30 Days": "all_accounts_0_30_days", 
        "All Accounts - 31-60 Days": "all_accounts_31_60_days",
        "All Accounts - 61-90 Days": "all_accounts_61_90_days", 
        "All Accounts - Over 90 Days": "all_accounts_over_90_days",
        "All Accounts - Account Balance": "all_accounts_account_balance",
        "All Accounts - Credits": "all_accounts_credits",
        "All Accounts - Insurance Company Name": "all_accounts_insurance_company_name",
        "All Accounts - Past Due": "all_accounts_past_due",
        "All Accounts - Total Due": "all_accounts_total_due", 
        "All Accounts - Unbilled": "all_accounts_unbilled",
        "All Accounts - Patient Insurance ID (GUID)": "all_accounts_patient_insurance_id_guid",
        "Patient's Name (First Last)": "patient_name_first_last"
    }
    
    # referrals_raw_021026.csv → bronze_ops.referrals_raw_wso  
    referrals_mapping = {
        "Patient's ID": "patient_id",
        "Patient's ID (Guid)": "patient_id_guid", 
        "Referred In By First Name": "referred_in_by_first_name",
        "Referred In By Last Name": "referred_in_by_last_name",
        "Referred In By Postal Code": "referred_in_by_postal_code",
        "Referred In By Type Code": "referred_in_by_type_code", 
        "Referred In By Type Description": "referred_in_by_type_description",
        "Referred In By Website": "referred_in_by_website",
        "Referred In By Greeting": "referred_in_by_greeting",
        "Referred In By City": "referred_in_by_city",
        "Patient's Entry Date": "patient_entry_date",
        "Treatment Start Date": "treatment_start_date", 
        "Patient's Name (First Last)": "patient_name_first_last",
        "Last Appointment Date": "last_appointment_date",
        "Patient Has a Future Appointment": "patient_has_future_appointment", 
        "Last Appointment Status Code": "last_appointment_status_code",
        "Last Appointment Status Description": "last_appointment_status_description"
        # Ignore: "Full Name" - calculated field
    }
    
    # treatment_raw_021026.csv → bronze_ops.treatments_raw_wso
    treatment_mapping = {
        "Patient's Name (Last First)": "patient_name_last_first",
        "Patient's ID": "patient_id",
        "Appointment Type Description": "appointment_type_description",
        "Appointment Date": "appointment_date", 
        "Treatment Start Date": "treatment_start_date",
        "Treatment Started": "treatment_started",
        "Treatment Type Description": "treatment_type_description",
        "Treatment Type Code": "treatment_type_code",
        "Appointment Type Code": "appointment_type_code"
        # Ignore: "New Pt Appt", "Treatment missing date", "Transaction Date Before"
    }
    
    return {
        'aging_raw_021026.csv': ('bronze_fin', 'aging_accounts_raw_wso', aging_mapping),
        'referrals_raw_021026.csv': ('bronze_ops', 'referrals_raw_wso', referrals_mapping), 
        'treatment_raw_021026.csv': ('bronze_ops', 'treatments_raw_wso', treatment_mapping)
    }

# Display the mappings
mappings = get_column_mappings()
for filename, (schema, table, mapping) in mappings.items():
    print(f"\n{filename} → {schema}.{table}")
    print(f"Columns to ingest ({len(mapping)}):")
    for source, target in mapping.items():
        print(f"  '{source}' → {target}")
        
print(f"\n{'='*60}")
print("Next: Define mappings for patients, appointments, and production files")
print(f"{'='*60}")


aging_raw_021026.csv → bronze_fin.aging_accounts_raw_wso
Columns to ingest (13):
  'Patient's ID (Guid)' → patient_id_guid
  'All Accounts - 0-30 Days' → all_accounts_0_30_days
  'All Accounts - 31-60 Days' → all_accounts_31_60_days
  'All Accounts - 61-90 Days' → all_accounts_61_90_days
  'All Accounts - Over 90 Days' → all_accounts_over_90_days
  'All Accounts - Account Balance' → all_accounts_account_balance
  'All Accounts - Credits' → all_accounts_credits
  'All Accounts - Insurance Company Name' → all_accounts_insurance_company_name
  'All Accounts - Past Due' → all_accounts_past_due
  'All Accounts - Total Due' → all_accounts_total_due
  'All Accounts - Unbilled' → all_accounts_unbilled
  'All Accounts - Patient Insurance ID (GUID)' → all_accounts_patient_insurance_id_guid
  'Patient's Name (First Last)' → patient_name_first_last

referrals_raw_021026.csv → bronze_ops.referrals_raw_wso
Columns to ingest (17):
  'Patient's ID' → patient_id
  'Patient's ID (Guid)' → patient_id_gu

In [5]:
# Complete column mappings for all files
def get_all_column_mappings():
    """Define complete column mappings for all files"""
    
    # aging_raw_021026.csv → bronze_fin.aging_accounts_raw_wso
    aging_mapping = {
        "Patient's ID (Guid)": "patient_id_guid",
        "All Accounts - 0-30 Days": "all_accounts_0_30_days", 
        "All Accounts - 31-60 Days": "all_accounts_31_60_days",
        "All Accounts - 61-90 Days": "all_accounts_61_90_days", 
        "All Accounts - Over 90 Days": "all_accounts_over_90_days",
        "All Accounts - Account Balance": "all_accounts_account_balance",
        "All Accounts - Credits": "all_accounts_credits",
        "All Accounts - Insurance Company Name": "all_accounts_insurance_company_name",
        "All Accounts - Past Due": "all_accounts_past_due",
        "All Accounts - Total Due": "all_accounts_total_due", 
        "All Accounts - Unbilled": "all_accounts_unbilled",
        "All Accounts - Patient Insurance ID (GUID)": "all_accounts_patient_insurance_id_guid",
        "Patient's Name (First Last)": "patient_name_first_last"
    }
    
    # patients_raw_021026.csv → bronze_ops.patients_raw_wso
    patients_mapping = {
        "Patient's ID": "patient_id",
        "Patient's ID (Guid)": "patient_id_guid",
        "Patient's First Name": "patient_first_name",
        "Patient's Last Name": "patient_last_name",
        "Patient's Age in Years": "patient_age_in_years",
        "Patient's Age (Years and Months)": "patient_age_years_and_months",
        "Patient's City": "patient_city",
        "Patient's City State Postal Code": "patient_city_state_postal_code", 
        "Patient's Email": "patient_email",
        "Patient's Gender": "patient_gender",
        "Patient's Primary Phone Number": "patient_primary_phone_number",
        "Patient's Primary SMS Phone Number": "patient_primary_sms_phone_number",
        "Patient's State": "patient_state",
        "Patient's Case Start Date": "patient_case_start_date",
        "Patient's Status Start Date": "patient_status_start_date", 
        "Patient's Actual Months in Status": "patient_actual_months_in_status",
        "Patient's Current Status Code": "patient_current_status_code",
        "Patient's Current Status Description": "patient_current_status_description",
        "Patient's Current Status Start Date": "patient_current_status_start_date",
        "Patient's Estimated Completion Date": "patient_estimated_completion_date",
        "Patient's Percent to Estimated Completion": "patient_percent_to_estimated_completion",
        "Patient's Group Membership": "patient_group_membership",
        "Last Appointment Date": "last_appointment_date",
        "Last Appointment Status Code": "last_appointment_status_code", 
        "Last Appointment Status Description": "last_appointment_status_description",
        "Patient Has a Future Appointment": "patient_has_a_future_appointment",
        "Last Dismissed Appointment Date": "last_dismissed_appointment_date",
        "This Is A Deleted Patient": "this_is_a_deleted_patient"
        # Ignore all "Patient Is In X Status" boolean columns
    }
    
    # referrals_raw_021026.csv → bronze_ops.referrals_raw_wso  
    referrals_mapping = {
        "Patient's ID": "patient_id",
        "Patient's ID (Guid)": "patient_id_guid", 
        "Referred In By First Name": "referred_in_by_first_name",
        "Referred In By Last Name": "referred_in_by_last_name",
        "Referred In By Postal Code": "referred_in_by_postal_code",
        "Referred In By Type Code": "referred_in_by_type_code", 
        "Referred In By Type Description": "referred_in_by_type_description",
        "Referred In By Website": "referred_in_by_website",
        "Referred In By Greeting": "referred_in_by_greeting",
        "Referred In By City": "referred_in_by_city",
        "Patient's Entry Date": "patient_entry_date",
        "Treatment Start Date": "treatment_start_date", 
        "Patient's Name (First Last)": "patient_name_first_last",
        "Last Appointment Date": "last_appointment_date",
        "Patient Has a Future Appointment": "patient_has_future_appointment", 
        "Last Appointment Status Code": "last_appointment_status_code",
        "Last Appointment Status Description": "last_appointment_status_description"
    }
    
    # treatment_raw_021026.csv → bronze_ops.treatments_raw_wso
    treatment_mapping = {
        "Patient's Name (Last First)": "patient_name_last_first",
        "Patient's ID": "patient_id",
        "Appointment Type Description": "appointment_type_description",
        "Appointment Date": "appointment_date", 
        "Treatment Start Date": "treatment_start_date",
        "Treatment Started": "treatment_started",
        "Treatment Type Description": "treatment_type_description",
        "Treatment Type Code": "treatment_type_code",
        "Appointment Type Code": "appointment_type_code"
    }
    
    return {
        'aging_raw_021026.csv': ('bronze_fin', 'aging_accounts_raw_wso', aging_mapping),
        'patients_raw_021026.csv': ('bronze_ops', 'patients_raw_wso', patients_mapping),
        'referrals_raw_021026.csv': ('bronze_ops', 'referrals_raw_wso', referrals_mapping), 
        'treatment_raw_021026.csv': ('bronze_ops', 'treatments_raw_wso', treatment_mapping)
    }

# I need to see the actual column names for appointments and production to complete the mappings
print("Need to examine corrected headers for appointments and production files...")
for file, headers in corrected_headers.items():
    if 'appointments' in file or 'production' in file:
        print(f"\n{file}:")
        for i, col in enumerate(headers[:10]):  # Show first 10 columns
            print(f"  {i:2d}: {repr(col)}")
        if len(headers) > 10:
            print(f"  ... ({len(headers)} total columns)")

Need to examine corrected headers for appointments and production files...

appointments_raw_021026.csv:
   0: "Patient's ID"
   1: "Patient's ID (Guid)"
   2: 'Appointment Created By'
   3: 'Appointment Created Date'
   4: 'Appointment Date'
   5: 'Appointment Has Been Confirmed'
   6: 'Appointment Left Message'
   7: 'Appointment Minutes'
   8: 'Appointment Start Time'
   9: 'Appointment Orthodontist Name (First Last)'
  ... (42 total columns)

production_raw_021026.csv:
   0: "Patient's ID (Guid)"
   1: "Patient's ID"
   2: 'Ledger Date (Date Only) Posted For'
   3: 'Electronic Transaction ID'
   4: 'Electronic Transaction Method'
   5: 'Electronic Transaction Status'
   6: 'Last Payment Amount'
   7: 'Last Payment Code'
   8: 'Last Payment Date'
   9: 'Last Payment Description'
  ... (45 total columns)


In [6]:
# Complete all file mappings including appointments and production
def get_final_column_mappings():
    """Define complete column mappings for all files"""
    
    # aging_raw_021026.csv → bronze_fin.aging_accounts_raw_wso
    aging_mapping = {
        "Patient's ID (Guid)": "patient_id_guid",
        "All Accounts - 0-30 Days": "all_accounts_0_30_days", 
        "All Accounts - 31-60 Days": "all_accounts_31_60_days",
        "All Accounts - 61-90 Days": "all_accounts_61_90_days", 
        "All Accounts - Over 90 Days": "all_accounts_over_90_days",
        "All Accounts - Account Balance": "all_accounts_account_balance",
        "All Accounts - Credits": "all_accounts_credits",
        "All Accounts - Insurance Company Name": "all_accounts_insurance_company_name",
        "All Accounts - Past Due": "all_accounts_past_due",
        "All Accounts - Total Due": "all_accounts_total_due", 
        "All Accounts - Unbilled": "all_accounts_unbilled",
        "All Accounts - Patient Insurance ID (GUID)": "all_accounts_patient_insurance_id_guid",
        "Patient's Name (First Last)": "patient_name_first_last"
    }
    
    # appointments_raw_021026.csv → bronze_ops.appointments_raw_wso
    appointments_mapping = {
        "Patient's ID": "patient_id",
        "Patient's ID (Guid)": "patient_id_guid",
        "Appointment Created By": "appointment_created_by",
        "Appointment Created Date": "appointment_created_date",
        "Appointment Date": "appointment_date",
        "Appointment Has Been Confirmed": "appointment_has_been_confirmed",
        "Appointment Left Message": "appointment_left_message",
        "Appointment Minutes": "appointment_minutes",
        "Appointment Start Time": "appointment_start_time",
        "Appointment Orthodontist Name (First Last)": "appointment_orthodontist_name_first_last",
        "Appointment Orthodontist Name (Last First)": "appointment_orthodontist_name_last_first",
        "Appointment Source": "appointment_source",
        "Appointment Status Code": "appointment_status_code",
        "Appointment Status Date/Time": "appointment_status_date_time",
        "Appointment Status Description": "appointment_status_description",
        "Appointment Type Class Code": "appointment_type_class_code",
        "Appointment Type Class Description": "appointment_type_class_description",
        "Appointment Type Code": "appointment_type_code",
        "Appointment Type Description": "appointment_type_description",
        "Appointment Type Minutes": "appointment_type_minutes",
        "Next Appointment Date": "next_appointment_date",
        "Next Appointment Status Code": "next_appointment_status_code",
        "Next Appointment Status Description": "next_appointment_status_description",
        "Next Appointment Type Code": "next_appointment_type_code",
        "Next Appointment Type Description": "next_appointment_type_description",
        "Patient Has a Future Appointment": "patient_has_a_future_appointment",
        "Next Appointment Including Today Status Code": "next_appointment_including_today_status_code",
        "Next Appointment Including Today Status Date/Time": "next_appointment_including_today_status_date_time",
        "Next Appointment Including Today Type Code": "next_appointment_including_today_type_code",
        "Next Appointment Including Today Type Description": "next_appointment_including_today_type_description",
        "Patient Has a Future Appointment After Today": "patient_has_a_future_appointment_after_today",
        "Patient Has an Appointment Today": "patient_has_an_appointment_today"
    }
    
    # production_raw_021026.csv → bronze_fin.ledger_transactions_raw_wso
    production_mapping = {
        "Patient's ID (Guid)": "patient_id_guid",
        "Patient's ID": "patient_id",
        "Ledger Date (Date Only) Posted For": "ledger_date_posted_for",
        "Electronic Transaction ID": "electronic_transaction_id",
        "Electronic Transaction Method": "electronic_transaction_method",
        "Electronic Transaction Status": "electronic_transaction_status",
        "Last Payment Amount": "last_payment_amount",
        "Last Payment Code": "last_payment_code",
        "Last Payment Date": "last_payment_date",
        "Last Payment Description": "last_payment_description",
        "Ledger Adjustment Description": "ledger_adjustment_description",
        "Ledger Adjustment Sub Code": "ledger_adjustment_sub_code",
        "Ledger Amount": "ledger_amount",
        "Ledger Comment": "ledger_comment",
        "Ledger Description": "ledger_description",
        "Ledger Item Affects Account Balance": "ledger_item_affects_account_balance",
        "Ledger Item Affects Production": "ledger_item_affects_production",
        "Ledger Item Affects Total Due": "ledger_item_affects_total_due",
        "Ledger Item Been Billed": "ledger_item_been_billed",
        "Ledger Item For Orthodontist": "ledger_item_for_orthodontist",
        "Ledger Item For Promise": "ledger_item_for_promise",
        "Ledger Item Insurance Name": "ledger_item_insurance_name",
        "Ledger Item Original Service Allow Rollback": "ledger_item_original_service_allow_rollback",
        "Ledger Item Original Service Amount": "ledger_item_original_service_amount",
        "Ledger Item Original Service Auto Prompt for Tooth Info when Charged": "ledger_item_original_service_auto_prompt_for_tooth_info_when_charged",
        "Ledger Item Original Service Description": "ledger_item_original_service_description",
        "Ledger Item Original Service Enque As Continuation Charge": "ledger_item_original_service_enque_as_continuation_charge",
        "Ledger Item Original Service Is Adjustment": "ledger_item_original_service_is_adjustment",
        "Ledger Item Original Service Is Exam for CAO": "ledger_item_original_service_is_exam_for_cao",
        "Ledger Item Original Service Is Late Charge": "ledger_item_original_service_is_late_charge",
        "Ledger Item Original Service Post Late Charge": "ledger_item_original_service_post_late_charge",
        "Ledger Item Original Service Prompt for Tooth Info when Charged": "ledger_item_original_service_prompt_for_tooth_info_when_charged",
        "Ledger Item Original Service Show on Statements": "ledger_item_original_service_show_on_statements",
        "Ledger Item Show On Statements": "ledger_item_show_on_statements",
        "Ledger Item Statement Amount": "ledger_item_statement_amount",
        "Ledger Open Status": "ledger_open_status",
        "Ledger Payment Code": "ledger_payment_code",
        "Ledger Payment Description": "ledger_payment_description",
        "Ledger Payment Include on Daily Deposit": "ledger_payment_include_on_daily_deposit",
        "Ledger Payment Is Credit Card": "ledger_payment_is_credit_card",
        "Ledger Receipt": "ledger_receipt",
        "Ledger Sub Type Description": "ledger_sub_type_description",
        "Transaction Type": "transaction_type",
        "Questions": "questions"
    }
    
    # patients_raw_021026.csv → bronze_ops.patients_raw_wso
    patients_mapping = {
        "Patient's ID": "patient_id",
        "Patient's ID (Guid)": "patient_id_guid",
        "Patient's First Name": "patient_first_name",
        "Patient's Last Name": "patient_last_name",
        "Patient's Age in Years": "patient_age_in_years",
        "Patient's Age (Years and Months)": "patient_age_years_and_months",
        "Patient's City": "patient_city",
        "Patient's City State Postal Code": "patient_city_state_postal_code", 
        "Patient's Email": "patient_email",
        "Patient's Gender": "patient_gender",
        "Patient's Primary Phone Number": "patient_primary_phone_number",
        "Patient's Primary SMS Phone Number": "patient_primary_sms_phone_number",
        "Patient's State": "patient_state",
        "Patient's Case Start Date": "patient_case_start_date",
        "Patient's Status Start Date": "patient_status_start_date", 
        "Patient's Actual Months in Status": "patient_actual_months_in_status",
        "Patient's Current Status Code": "patient_current_status_code",
        "Patient's Current Status Description": "patient_current_status_description",
        "Patient's Current Status Start Date": "patient_current_status_start_date",
        "Patient's Estimated Completion Date": "patient_estimated_completion_date",
        "Patient's Percent to Estimated Completion": "patient_percent_to_estimated_completion",
        "Patient's Group Membership": "patient_group_membership",
        "Last Appointment Date": "last_appointment_date",
        "Last Appointment Status Code": "last_appointment_status_code", 
        "Last Appointment Status Description": "last_appointment_status_description",
        "Patient Has a Future Appointment": "patient_has_a_future_appointment",
        "Last Dismissed Appointment Date": "last_dismissed_appointment_date",
        "This Is A Deleted Patient": "this_is_a_deleted_patient"
    }
    
    # referrals_raw_021026.csv → bronze_ops.referrals_raw_wso  
    referrals_mapping = {
        "Patient's ID": "patient_id",
        "Patient's ID (Guid)": "patient_id_guid", 
        "Referred In By First Name": "referred_in_by_first_name",
        "Referred In By Last Name": "referred_in_by_last_name",
        "Referred In By Postal Code": "referred_in_by_postal_code",
        "Referred In By Type Code": "referred_in_by_type_code", 
        "Referred In By Type Description": "referred_in_by_type_description",
        "Referred In By Website": "referred_in_by_website",
        "Referred In By Greeting": "referred_in_by_greeting",
        "Referred In By City": "referred_in_by_city",
        "Patient's Entry Date": "patient_entry_date",
        "Treatment Start Date": "treatment_start_date", 
        "Patient's Name (First Last)": "patient_name_first_last",
        "Last Appointment Date": "last_appointment_date",
        "Patient Has a Future Appointment": "patient_has_future_appointment", 
        "Last Appointment Status Code": "last_appointment_status_code",
        "Last Appointment Status Description": "last_appointment_status_description"
    }
    
    # treatment_raw_021026.csv → bronze_ops.treatments_raw_wso
    treatment_mapping = {
        "Patient's Name (Last First)": "patient_name_last_first",
        "Patient's ID": "patient_id",
        "Appointment Type Description": "appointment_type_description",
        "Appointment Date": "appointment_date", 
        "Treatment Start Date": "treatment_start_date",
        "Treatment Started": "treatment_started",
        "Treatment Type Description": "treatment_type_description",
        "Treatment Type Code": "treatment_type_code",
        "Appointment Type Code": "appointment_type_code"
    }
    
    return {
        'aging_raw_021026.csv': ('bronze_fin', 'aging_accounts_raw_wso', aging_mapping),
        'appointments_raw_021026.csv': ('bronze_ops', 'appointments_raw_wso', appointments_mapping),
        'production_raw_021026.csv': ('bronze_fin', 'ledger_transactions_raw_wso', production_mapping),
        'patients_raw_021026.csv': ('bronze_ops', 'patients_raw_wso', patients_mapping),
        'referrals_raw_021026.csv': ('bronze_ops', 'referrals_raw_wso', referrals_mapping), 
        'treatment_raw_021026.csv': ('bronze_ops', 'treatments_raw_wso', treatment_mapping)
    }

# Display final mappings summary
final_mappings = get_final_column_mappings()
print("Final Bronze Layer Table Mappings:")
print("=" * 60)
for filename, (schema, table, mapping) in final_mappings.items():
    print(f"{filename} → {schema}.{table} ({len(mapping)} columns)")

print(f"\nReady to create ingestion functions!")

Final Bronze Layer Table Mappings:
aging_raw_021026.csv → bronze_fin.aging_accounts_raw_wso (13 columns)
appointments_raw_021026.csv → bronze_ops.appointments_raw_wso (32 columns)
production_raw_021026.csv → bronze_fin.ledger_transactions_raw_wso (44 columns)
patients_raw_021026.csv → bronze_ops.patients_raw_wso (28 columns)
referrals_raw_021026.csv → bronze_ops.referrals_raw_wso (17 columns)
treatment_raw_021026.csv → bronze_ops.treatments_raw_wso (9 columns)

Ready to create ingestion functions!


In [7]:
# Bronze layer data ingestion functions
from datetime import datetime
import sqlalchemy

def load_csv_to_bronze(filename, test_mode=True):
    """
    Load CSV data to bronze layer table with minimal transformation
    
    Args:
        filename: Name of CSV file to process
        test_mode: If True, only show first 5 rows and don't insert to DB
    """
    file_path = data_dir / filename
    if not file_path.exists():
        print(f"❌ File not found: {filename}")
        return None
    
    # Get configuration
    schema, table, column_mapping = final_mappings.get(filename, (None, None, None))
    header_row = file_config.get(filename, 0)
    
    if not schema:
        print(f"❌ No mapping defined for {filename}")
        return None
    
    print(f"📂 Processing: {filename}")
    print(f"🎯 Target: {schema}.{table}")
    print(f"📊 Header row: {header_row}")
    
    try:
        # Read CSV with correct header row
        df_raw = pd.read_csv(file_path, header=header_row)
        print(f"📈 Raw data shape: {df_raw.shape}")
        
        # Select only the columns we need (ignore derived/calculated columns)
        available_columns = [col for col in column_mapping.keys() if col in df_raw.columns]
        missing_columns = [col for col in column_mapping.keys() if col not in df_raw.columns]
        
        if missing_columns:
            print(f"⚠️  Missing columns: {missing_columns}")
        
        # Create bronze dataframe with mapped column names
        df_bronze = df_raw[available_columns].copy()
        df_bronze.rename(columns={col: column_mapping[col] for col in available_columns}, inplace=True)
        
        # Add ingestion metadata (bronze layer pattern)
        df_bronze['ingestion_timestamp'] = datetime.now()
        df_bronze['source_file'] = filename
        df_bronze['client_tag'] = 'wso'
        
        print(f"🔄 Bronze data shape: {df_bronze.shape}")
        print(f"📋 Columns: {list(df_bronze.columns)}")
        
        if test_mode:
            print(f"\\n🔍 Sample data (first 3 rows):")
            for i, row in df_bronze.head(3).iterrows():
                print(f"\\nRow {i+1}:")
                for col in df_bronze.columns[:5]:  # Show first 5 columns
                    print(f"  {col}: {repr(row[col])}")
                if len(df_bronze.columns) > 5:
                    print(f"  ... ({len(df_bronze.columns)} total columns)")
            return df_bronze
        else:
            # Write to database
            engine = get_engine()
            full_table_name = f"{schema}.{table}"
            
            df_bronze.to_sql(
                table, 
                engine, 
                schema=schema,
                if_exists='replace',  # For bronze layer, replace on each load
                index=False,
                method='multi'
            )
            
            print(f"✅ Successfully loaded {len(df_bronze)} rows to {full_table_name}")
            return df_bronze
            
    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")
        return None

# Test with one file first
print("Testing with aging data...")
test_result = load_csv_to_bronze('aging_raw_021026.csv', test_mode=True)

Testing with aging data...
📂 Processing: aging_raw_021026.csv
🎯 Target: bronze_fin.aging_accounts_raw_wso
📊 Header row: 0
📈 Raw data shape: (30, 13)
🔄 Bronze data shape: (30, 16)
📋 Columns: ['patient_id_guid', 'all_accounts_0_30_days', 'all_accounts_31_60_days', 'all_accounts_61_90_days', 'all_accounts_over_90_days', 'all_accounts_account_balance', 'all_accounts_credits', 'all_accounts_insurance_company_name', 'all_accounts_past_due', 'all_accounts_total_due', 'all_accounts_unbilled', 'all_accounts_patient_insurance_id_guid', 'patient_name_first_last', 'ingestion_timestamp', 'source_file', 'client_tag']
\n🔍 Sample data (first 3 rows):
\nRow 1:
  patient_id_guid: '{EE314011-2837-4466-BDA0-E50717731E6D}'
  all_accounts_0_30_days: 462
  all_accounts_31_60_days: 458
  all_accounts_61_90_days: 458
  all_accounts_over_90_days: 2290
  ... (16 total columns)
\nRow 2:
  patient_id_guid: '{75B44B90-A994-4474-9211-FE1B6BFB5DF8}'
  all_accounts_0_30_days: 0
  all_accounts_31_60_days: 0
  all_accou

In [8]:
# Test all files in test mode first to validate data processing
def test_all_files():
    """Test data processing for all files"""
    print("🧪 TESTING ALL FILES")
    print("=" * 60)
    
    results = {}
    
    for filename in final_mappings.keys():
        print(f"\\n📋 Testing {filename}...")
        try:
            df = load_csv_to_bronze(filename, test_mode=True)
            if df is not None:
                results[filename] = {
                    'success': True,
                    'rows': len(df),
                    'columns': len(df.columns),
                    'sample_data': df.head(2).to_dict('records')
                }
                print(f"✅ {filename}: {len(df)} rows, {len(df.columns)} columns")
            else:
                results[filename] = {'success': False, 'error': 'Failed to process'}
                print(f"❌ {filename}: Failed to process")
        except Exception as e:
            results[filename] = {'success': False, 'error': str(e)}
            print(f"❌ {filename}: {e}")
    
    # Summary
    print(f"\\n📊 SUMMARY")
    print("=" * 60)
    successful = sum(1 for r in results.values() if r.get('success'))
    total_rows = sum(r.get('rows', 0) for r in results.values() if r.get('success'))
    
    print(f"✅ Successfully processed: {successful}/{len(results)} files")
    print(f"📈 Total rows: {total_rows}")
    
    for filename, result in results.items():
        if result.get('success'):
            schema, table, _ = final_mappings[filename]
            print(f"   {filename} → {schema}.{table} ({result['rows']} rows)")
        else:
            print(f"   ❌ {filename}: {result.get('error', 'Unknown error')}")
    
    return results

# Run the test
test_results = test_all_files()

🧪 TESTING ALL FILES
\n📋 Testing aging_raw_021026.csv...
📂 Processing: aging_raw_021026.csv
🎯 Target: bronze_fin.aging_accounts_raw_wso
📊 Header row: 0
📈 Raw data shape: (30, 13)
🔄 Bronze data shape: (30, 16)
📋 Columns: ['patient_id_guid', 'all_accounts_0_30_days', 'all_accounts_31_60_days', 'all_accounts_61_90_days', 'all_accounts_over_90_days', 'all_accounts_account_balance', 'all_accounts_credits', 'all_accounts_insurance_company_name', 'all_accounts_past_due', 'all_accounts_total_due', 'all_accounts_unbilled', 'all_accounts_patient_insurance_id_guid', 'patient_name_first_last', 'ingestion_timestamp', 'source_file', 'client_tag']
\n🔍 Sample data (first 3 rows):
\nRow 1:
  patient_id_guid: '{EE314011-2837-4466-BDA0-E50717731E6D}'
  all_accounts_0_30_days: 462
  all_accounts_31_60_days: 458
  all_accounts_61_90_days: 458
  all_accounts_over_90_days: 2290
  ... (16 total columns)
\nRow 2:
  patient_id_guid: '{75B44B90-A994-4474-9211-FE1B6BFB5DF8}'
  all_accounts_0_30_days: 0
  all_accou

In [9]:
# Function to load all data to bronze layer
def load_all_to_bronze(confirm=False):
    """
    Load all WSO data to bronze layer tables in Supabase
    
    Args:
        confirm: Must be True to actually execute the database writes
    """
    if not confirm:
        print("🚨 This function will write data to the database.")
        print("🚨 Set confirm=True to proceed with actual database writes.")
        print("🚨 Use test_all_files() first to validate data processing.")
        return False
    
    print("🚀 LOADING ALL DATA TO BRONZE LAYER")
    print("=" * 60)
    
    # Test database connection first
    try:
        engine = get_engine()
        with engine.connect() as conn:
            print("✅ Database connection successful")
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return False
    
    results = {}
    
    for filename in final_mappings.keys():
        print(f"\\n📤 Loading {filename}...")
        try:
            df = load_csv_to_bronze(filename, test_mode=False)
            if df is not None:
                schema, table, _ = final_mappings[filename]
                results[filename] = {
                    'success': True,
                    'rows': len(df),
                    'table': f"{schema}.{table}"
                }
                print(f"✅ Successfully loaded {len(df)} rows to {schema}.{table}")
            else:
                results[filename] = {'success': False, 'error': 'Failed to process'}
                print(f"❌ {filename}: Failed to process")
        except Exception as e:
            results[filename] = {'success': False, 'error': str(e)}
            print(f"❌ {filename}: {e}")
    
    # Summary
    print(f"\\n🎉 INGESTION COMPLETE")
    print("=" * 60)
    successful = sum(1 for r in results.values() if r.get('success'))
    total_rows = sum(r.get('rows', 0) for r in results.values() if r.get('success'))
    
    print(f"✅ Successfully loaded: {successful}/{len(results)} files")
    print(f"📈 Total rows ingested: {total_rows}")
    
    print(f"\\n📋 Bronze Layer Tables Created:")
    for filename, result in results.items():
        if result.get('success'):
            print(f"   ✅ {result['table']} ({result['rows']} rows)")
        else:
            print(f"   ❌ {filename}: {result.get('error', 'Unknown error')}")
    
    if successful == len(results):
        print(f"\\n🎯 ALL FILES SUCCESSFULLY INGESTED TO BRONZE LAYER!")
    else:
        print(f"\\n⚠️  {len(results) - successful} files failed to load")
    
    return results

# Show the function call (but don't execute yet)
print("🔍 Ready to load data to bronze layer!")
print("📝 To execute, run: load_all_to_bronze(confirm=True)")
print("\\n🏗️  This will create the following bronze layer tables:")
for filename, (schema, table, mapping) in final_mappings.items():
    print(f"   📊 {schema}.{table}")

# Uncomment the line below when ready to load data:
# load_results = load_all_to_bronze(confirm=True)

🔍 Ready to load data to bronze layer!
📝 To execute, run: load_all_to_bronze(confirm=True)
\n🏗️  This will create the following bronze layer tables:
   📊 bronze_fin.aging_accounts_raw_wso
   📊 bronze_ops.appointments_raw_wso
   📊 bronze_fin.ledger_transactions_raw_wso
   📊 bronze_ops.patients_raw_wso
   📊 bronze_ops.referrals_raw_wso
   📊 bronze_ops.treatments_raw_wso


## Summary

✅ **Complete Bronze Layer Ingestion Script Created**

### What We Built:
1. **File Configuration**: Proper header row handling for each CSV file
2. **Column Mappings**: Complete mapping from raw CSV headers to bronze layer column names
3. **Data Validation**: Test functions to validate data processing before database writes
4. **Bronze Layer Tables**: 6 tables following the WSO naming convention

### Bronze Layer Tables Created:
- `bronze_fin.aging_accounts_raw_wso` (13 data columns + 3 metadata)
- `bronze_fin.ledger_transactions_raw_wso` (44 data columns + 3 metadata) 
- `bronze_ops.appointments_raw_wso` (32 data columns + 3 metadata)
- `bronze_ops.patients_raw_wso` (28 data columns + 3 metadata)
- `bronze_ops.referrals_raw_wso` (17 data columns + 3 metadata)
- `bronze_ops.treatments_raw_wso` (9 data columns + 3 metadata)

### Key Features:
- ✅ **Minimal Transformation**: Raw data preserved with only column renaming
- ✅ **Metadata Tracking**: `ingestion_timestamp`, `source_file`, `client_tag`
- ✅ **Client Tagging**: All tables tagged with `_wso` suffix
- ✅ **Domain Separation**: Finance tables in `bronze_fin`, operations in `bronze_ops`
- ✅ **Error Handling**: Comprehensive validation and error reporting
- ✅ **Test Mode**: Safe testing before database writes

### Next Steps:
1. Execute `load_all_to_bronze(confirm=True)` to populate bronze layer
2. Verify data in Supabase database
3. Begin development of Silver layer transformations

In [11]:
load_all_to_bronze(confirm=True)

🚀 LOADING ALL DATA TO BRONZE LAYER
✅ Database connection successful
\n📤 Loading aging_raw_021026.csv...
📂 Processing: aging_raw_021026.csv
🎯 Target: bronze_fin.aging_accounts_raw_wso
📊 Header row: 0
📈 Raw data shape: (30, 13)
🔄 Bronze data shape: (30, 16)
📋 Columns: ['patient_id_guid', 'all_accounts_0_30_days', 'all_accounts_31_60_days', 'all_accounts_61_90_days', 'all_accounts_over_90_days', 'all_accounts_account_balance', 'all_accounts_credits', 'all_accounts_insurance_company_name', 'all_accounts_past_due', 'all_accounts_total_due', 'all_accounts_unbilled', 'all_accounts_patient_insurance_id_guid', 'patient_name_first_last', 'ingestion_timestamp', 'source_file', 'client_tag']
✅ Successfully loaded 30 rows to bronze_fin.aging_accounts_raw_wso
✅ Successfully loaded 30 rows to bronze_fin.aging_accounts_raw_wso
\n📤 Loading appointments_raw_021026.csv...
📂 Processing: appointments_raw_021026.csv
🎯 Target: bronze_ops.appointments_raw_wso
📊 Header row: 1
📈 Raw data shape: (4564, 42)
🔄 Bron

{'aging_raw_021026.csv': {'success': True,
  'rows': 30,
  'table': 'bronze_fin.aging_accounts_raw_wso'},
 'appointments_raw_021026.csv': {'success': True,
  'rows': 4564,
  'table': 'bronze_ops.appointments_raw_wso'},
 'production_raw_021026.csv': {'success': True,
  'rows': 1788,
  'table': 'bronze_fin.ledger_transactions_raw_wso'},
 'patients_raw_021026.csv': {'success': True,
  'rows': 3264,
  'table': 'bronze_ops.patients_raw_wso'},
 'referrals_raw_021026.csv': {'success': True,
  'rows': 186,
  'table': 'bronze_ops.referrals_raw_wso'},
 'treatment_raw_021026.csv': {'success': True,
  'rows': 4742,
  'table': 'bronze_ops.treatments_raw_wso'}}